# Inputs

In [3]:
# Path to chromedriver. Use same \ and keep leading r
chromedriver_path = r"C:\Users\Alessandro Bitetto\Downloads\UniPV\Scraping_PEC\WebDriver\chromedriver"

# web page
web_page = "https://www.ufficiocamerale.it/cerca-pec-azienda"

# VAT list  - VAT must be without leading "IT" - .csv with 1 column "VAT" - Use same \ and keep leading r
VAT_file = r"C:\Users\Alessandro Bitetto\Downloads\110Laude\110L_SIR\03_Output\df_VAR_for_PEC.csv"
VAT_file_sep = ";"

# output csv separator
OUT_file_sep = ";"

# Scrape

In [ ]:
from utils import get_chromedriver, scrape_PEC

scrape_PEC(chromedriver_path, web_page, VAT_file, VAT_file_sep, OUT_file_sep)

Launching Chrome...OK

